Skeleton Code

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()
print(df.head())

# Summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Data type information
print(df.info())
#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


        0         1         2         3         4         5         6    \
0  0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1  0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2  0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3  0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4  0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   

        7         8         9    ...       551       552       553       554  \
0 -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304 -0.112754   
1 -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499  0.053477   
2 -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104 -0.118559   
3 -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845 -0.036788   
4 -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205  0.123320   

        555       556       557       558       559       560  
0  0

In [3]:
# Task 2: Encode class labels
# YOUR CODE HERE: Use LabelEncoder to encode class labels

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df_scaled=scaler.fit_transform(df)

In [9]:
# Task 4: Split the data into training and testing sets
# YOUR CODE HERE: Use train_test_split to split the data
from sklearn.model_selection import train_test_split
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled,encoded_y,test_size=0.2,random_state=42)

In [10]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()

In [13]:
start_time=time.time()
pipeline=Pipeline([('model',GaussianNB())])
pipeline.fit(X_train_full,y_train)
ypred=pipeline.predict(X_test_full)
end_time=time.time()
time_taken=end_time-start_time
accuracy=accuracy_score(y_test,ypred)

In [14]:
print("Accuracy Score:",accuracy)
print("Time taken:",time_taken)

Accuracy Score: 0.7314751869476547
Time taken: 0.16876792907714844


In [16]:
# TASK 7 - K-Means for dimensionality reduction
from sklearn.cluster import KMeans
import numpy as np
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters,random_state=42)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices =np.argmin(kmeans.transform(df_scaled.T),axis=0)
selected_features = df_scaled[:,selected_features_indices]

In [17]:
print("selected_features_indices:",selected_features_indices)
print("Selected features:",selected_features)

selected_features_indices: [210 266  12 541 260 453 513  66  45 280   5 539  42 470  65 121 177 189
  96 499  56 538  98 241 178  50 456 153 526  27  31  74 162 329 150 160
 147  46 458 158 341  38   0 380 156 209  81 110 142 188]
Selected features: [[-0.20070185 -0.91788977  0.90600869 ... -0.53409272  0.50214028
  -1.03215662]
 [-0.56330593 -0.90521141  0.89599818 ... -0.00671888  0.3376477
  -0.64116671]
 [-0.41589069 -0.89623812  0.88032325 ... -0.31600032 -0.24977142
  -0.40758844]
 ...
 [ 1.83768483  0.60550849 -0.71616571 ... -0.31836518  0.08603614
  -0.29794173]
 [ 1.7847828   0.47993785 -0.79407448 ...  0.24976279 -1.00295814
  -0.67163312]
 [ 1.53001801  0.65873714 -0.79407448 ...  1.17998565  0.51074335
  -0.40849264]]


In [19]:
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df_scaled[:, selected_features_indices]

In [20]:
print("selected_features_indices:",selected_features_indices)
print("Selected features:",selected_features)

selected_features_indices: [155, 15, 141, 31, 539, 21, 59, 211, 52, 475, 477, 42, 176, 444, 172, 298, 249, 527, 555, 238, 291, 380, 82, 360, 123, 116, 362, 277, 498, 41, 189, 68, 118, 240, 158, 72, 37, 80, 157, 108, 328, 301, 152, 161, 212, 346, 36, 459, 25, 261]
Selected features: [[ 2.57602421 -0.885916   -0.84576709 ... -0.71840922  3.41252516
   0.05426879]
 [ 1.100926   -0.92069378 -0.83736443 ...  0.24525519  2.37604697
  -0.28975396]
 [ 0.51449745 -0.92397428 -0.8268538  ... -0.67367648  1.27125913
   0.19924571]
 ...
 [-1.87078992  1.17768286  0.77398378 ...  0.4560065  -0.8488946
  -0.13816968]
 [-2.53817908  1.20307032  0.55595075 ...  0.23656391 -1.16992694
  -1.17094764]
 [-1.82848539  1.10139308  1.14297738 ...  0.15363336 -1.06466879
  -1.00669305]]


In [21]:
X_train_full, X_test_full, y_train, y_test = train_test_split(selected_features,encoded_y,test_size=0.2,random_state=42)

In [22]:
model1=GaussianNB()
start=time.time()
model1.fit(X_train_full,y_train)
y_pred=model1.predict(X_test_full)
end=time.time()
new_time_taken=end-start
new_accuracy=accuracy_score(y_test,y_pred)


In [23]:
print("New accuracy:",new_accuracy)
print("New time taken:",new_time_taken)


New accuracy: 0.8592794017675051
New time taken: 0.04877209663391113
